01_LGBM_baseline.ipynb와 동일한 feature + 추가 6개

배아_생성_효율 |
배아_이식_비율 |
배아_저장_비율 |
나이xDay5 |
시술횟수x나이 |
품질_점수

LGBM + Catboost + XGboost stacking 

In [1]:
import lightgbm as lgb
import catboost as cb
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
import warnings
import pandas as pd
import numpy as np
warnings.filterwarnings('ignore')

# 1. 데이터 로드
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")
submission = pd.read_csv("../data/sample_submission.csv")

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print(f"Submission shape: {submission.shape}")

# 2. 전처리 함수 (train, test 동일하게 적용)
def preprocess(df):
    df_copy = df.copy()
    
    # 시술_대분류
    def major_procedure(x):
        if pd.isna(x):
            return "Unknown"
        if "IUI" in x:
            return "IUI"
        if "DI" in x:
            return "Other"
        if "ICSI" in x:
            return "ICSI"
        if "IVF" in x:
            return "IVF"
        return "Other"
    
    df_copy["시술_대분류"] = df_copy["특정 시술 유형"].apply(major_procedure)
    
    # BLASTOCYST 포함 여부
    df_copy["BLASTOCYST_포함"] = df_copy["특정 시술 유형"].str.contains("BLASTOCYST", na=False).astype(int)
    
    # 배아 이식 여부 (21개 컬럼 전부 체크)
    embryo_stage_cols = [
        "단일 배아 이식 여부",
        "착상 전 유전 진단 사용 여부",
        "배아 생성 주요 이유",
        "총 생성 배아 수",
        "미세주입된 난자 수",
        "미세주입에서 생성된 배아 수",
        "이식된 배아 수",
        "미세주입 배아 이식 수",
        "저장된 배아 수",
        "미세주입 후 저장된 배아 수",
        "해동된 배아 수",
        "해동 난자 수",
        "수집된 신선 난자 수",
        "저장된 신선 난자 수",
        "혼합된 난자 수",
        "파트너 정자와 혼합된 난자 수",
        "기증자 정자와 혼합된 난자 수",
        "동결 배아 사용 여부",
        "신선 배아 사용 여부",
        "기증 배아 사용 여부",
        "대리모 여부",
    ]
    
    df_copy["배아_이식_미도달"] = df_copy[embryo_stage_cols].isna().all(axis=1).astype(int)
    df_copy["배아_이식_여부"] = 1 - df_copy["배아_이식_미도달"]
    
    # 배아 진행 단계 (NEW!)
    def embryo_stage(row):
        if row['배아_이식_여부'] == 0:
            return '배아단계_미도달'
        elif pd.isna(row['총 생성 배아 수']) or row['총 생성 배아 수'] == 0:
            return '배아생성_실패'
        elif pd.isna(row['이식된 배아 수']) or row['이식된 배아 수'] == 0:
            return '이식_미실시'
        else:
            return '이식_완료'
    
    df_copy['배아_진행_단계'] = df_copy.apply(embryo_stage, axis=1)
    
    # 총시술_bin3
    def collapse_trials(x):
        if x == '0회':
            return '0회'
        elif x in ['1회', '2회']:
            return '1–2회'
        else:
            return '3회 이상'
    
    df_copy["총시술_bin3"] = df_copy["총 시술 횟수"].apply(collapse_trials)
    
    # 나이_3구간
    def age_group_simple(age):
        if age == '알 수 없음':
            return 'Unknown'
        elif age == '만18-34세':
            return '34세 이하'
        elif age in ['만35-37세', '만38-39세']:
            return '35-39세'
        else:
            return '40세 이상'
    
    df_copy['나이_3구간'] = df_copy['시술 당시 나이'].apply(age_group_simple)
    
    # 이식배아_구간
    def embryo_count_bin(count):
        if pd.isna(count) or count == 0:
            return '0개'
        elif count <= 2:
            return '1-2개'
        else:
            return '3개 이상'
    
    df_copy['이식배아_구간'] = df_copy['이식된 배아 수'].apply(embryo_count_bin)
    
    # Day5_이식_여부
    df_copy['Day5_이식_여부'] = (df_copy['배아 이식 경과일'] == 5.0).astype(int)
    
    # 불임원인_복잡도
    infertility_cols = [
        "남성 주 불임 원인", "남성 부 불임 원인", "여성 주 불임 원인", "여성 부 불임 원인",
        "부부 주 불임 원인", "부부 부 불임 원인", "불명확 불임 원인",
        "불임 원인 - 난관 질환", "불임 원인 - 남성 요인", "불임 원인 - 배란 장애",
        "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증",
        "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성",
        "불임 원인 - 정자 형태"
    ]
    
    df_copy["불임_원인_개수"] = df_copy[infertility_cols].sum(axis=1)
    
    def infertility_complexity(count):
        if count == 0:
            return 'None'
        elif count == 1:
            return 'Single'
        elif count == 2:
            return 'Double'
        else:
            return 'Multiple'
    
    df_copy['불임원인_복잡도'] = df_copy['불임_원인_개수'].apply(infertility_complexity)
    
    # 배아_해동_실시_여부
    df_copy['배아_해동_실시_여부'] = df_copy['배아 해동 경과일'].notna().astype(int)
    
    # === 새로 추가: 배아 효율 비율 변수들 (NEW!) ===
    df_copy['배아_생성_효율'] = df_copy['총 생성 배아 수'] / (df_copy['수집된 신선 난자 수'] + 1)
    df_copy['배아_이식_비율'] = df_copy['이식된 배아 수'] / (df_copy['총 생성 배아 수'] + 1)
    df_copy['배아_저장_비율'] = df_copy['저장된 배아 수'] / (df_copy['총 생성 배아 수'] + 1)
    
    # === 새로 추가: 교호작용 변수들 (NEW!) ===
    df_copy['나이×Day5'] = df_copy['시술 당시 나이'].astype(str) + '_' + df_copy['Day5_이식_여부'].astype(str)
    df_copy['시술횟수×나이'] = df_copy['총시술_bin3'] + '_' + df_copy['나이_3구간']
    
    return df_copy

# 3. 전처리 적용
train_df = preprocess(train_df)
test_df = preprocess(test_df)

print(f"\n전처리 후 Train shape: {train_df.shape}")
print(f"전처리 후 Test shape: {test_df.shape}")

# 4. Feature 준비
feature_cols = [col for col in train_df.columns if col not in ['ID', '임신 성공 여부', '배아_이식_미도달']]

X_train = train_df[feature_cols].copy()
y_train = train_df['임신 성공 여부'].copy()
X_test = test_df[feature_cols].copy()

# 5. 범주형 변수 지정
categorical_features = [
    '시술 시기 코드', '시술 당시 나이', '시술 유형', '특정 시술 유형', '배란 유도 유형',
    '배아 생성 주요 이유', '총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수',
    'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수',
    'IVF 출산 횟수', 'DI 출산 횟수', '난자 출처', '정자 출처', '난자 기증자 나이',
    '정자 기증자 나이', '시술_대분류', '총시술_bin3', '나이_3구간', '이식배아_구간',
    '불임원인_복잡도', '배아_진행_단계', '나이×Day5', '시술횟수×나이',  # NEW!
]

# 범주형 변수 인덱스 (CatBoost, XGBoost용)
cat_features_idx = [i for i, col in enumerate(feature_cols) if col in categorical_features]

# LightGBM용 category 타입 변환
X_train_lgb = X_train.copy()
X_test_lgb = X_test.copy()
for col in categorical_features:
    if col in X_train_lgb.columns:
        X_train_lgb[col] = X_train_lgb[col].astype('category')
        X_test_lgb[col] = X_test_lgb[col].astype('category')

# CatBoost, XGBoost용 Label Encoding
X_train_encoded = X_train.copy()
X_test_encoded = X_test.copy()

from sklearn.preprocessing import LabelEncoder
label_encoders = {}

for col in categorical_features:
    if col in X_train_encoded.columns:
        le = LabelEncoder()
        # train + test 합쳐서 fit (동일한 카테고리 보장)
        all_values = pd.concat([X_train_encoded[col], X_test_encoded[col]]).astype(str)
        le.fit(all_values)
        X_train_encoded[col] = le.transform(X_train_encoded[col].astype(str))
        X_test_encoded[col] = le.transform(X_test_encoded[col].astype(str))
        label_encoders[col] = le

print(f"\n범주형 변수 {len(categorical_features)}개 인코딩 완료")

# ============================
# 6. StratifiedKFold 설정
# ============================
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# OOF 예측 저장용
oof_lgb = np.zeros(len(X_train))
oof_cat = np.zeros(len(X_train))
oof_xgb = np.zeros(len(X_train))

# Test 예측 저장용
test_lgb = np.zeros(len(X_test))
test_cat = np.zeros(len(X_test))
test_xgb = np.zeros(len(X_test))

# ============================
# 7. Model 1: LightGBM
# ============================
print("\n" + "="*60)
print("MODEL 1: LightGBM")
print("="*60)

lgb_params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'random_state': 42,
}

for fold, (train_idx, valid_idx) in enumerate(skf.split(X_train_lgb, y_train), 1):
    print(f"\n--- Fold {fold} ---")
    
    X_tr, X_val = X_train_lgb.iloc[train_idx], X_train_lgb.iloc[valid_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]
    
    train_data = lgb.Dataset(X_tr, label=y_tr, categorical_feature=categorical_features)
    valid_data = lgb.Dataset(X_val, label=y_val, categorical_feature=categorical_features, reference=train_data)
    
    model = lgb.train(
        lgb_params,
        train_data,
        num_boost_round=1000,
        valid_sets=[train_data, valid_data],
        valid_names=['train', 'valid'],
        callbacks=[
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(period=200)
        ]
    )
    
    oof_lgb[valid_idx] = model.predict(X_val, num_iteration=model.best_iteration)
    test_lgb += model.predict(X_test_lgb, num_iteration=model.best_iteration) / skf.n_splits
    
    fold_auc = roc_auc_score(y_val, oof_lgb[valid_idx])
    print(f"Fold {fold} AUC: {fold_auc:.6f}")

lgb_cv_auc = roc_auc_score(y_train, oof_lgb)
print(f"\n✅ LightGBM CV AUC: {lgb_cv_auc:.6f}")

# ============================
# 8. Model 2: CatBoost
# ============================
print("\n" + "="*60)
print("MODEL 2: CatBoost")
print("="*60)

for fold, (train_idx, valid_idx) in enumerate(skf.split(X_train_encoded, y_train), 1):
    print(f"\n--- Fold {fold} ---")
    
    X_tr, X_val = X_train_encoded.iloc[train_idx], X_train_encoded.iloc[valid_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]
    
    model = cb.CatBoostClassifier(
        iterations=1000,
        learning_rate=0.05,
        depth=8,
        loss_function='Logloss',
        eval_metric='AUC',
        cat_features=cat_features_idx,
        random_seed=42,
        early_stopping_rounds=50,
        verbose=200
    )
    
    model.fit(
        X_tr, y_tr,
        eval_set=(X_val, y_val),
        use_best_model=True,
        verbose=False
    )
    
    oof_cat[valid_idx] = model.predict_proba(X_val)[:, 1]
    test_cat += model.predict_proba(X_test_encoded)[:, 1] / skf.n_splits
    
    fold_auc = roc_auc_score(y_val, oof_cat[valid_idx])
    print(f"Fold {fold} AUC: {fold_auc:.6f}")

cat_cv_auc = roc_auc_score(y_train, oof_cat)
print(f"\n✅ CatBoost CV AUC: {cat_cv_auc:.6f}")

# ============================
# 9. Model 3: XGBoost
# ============================
print("\n" + "="*60)
print("MODEL 3: XGBoost")
print("="*60)

xgb_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'max_depth': 8,
    'learning_rate': 0.05,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight': 1,
    'reg_alpha': 0.1,
    'reg_lambda': 1.0,
    'random_state': 42,
    'tree_method': 'hist',
}

for fold, (train_idx, valid_idx) in enumerate(skf.split(X_train_encoded, y_train), 1):
    print(f"\n--- Fold {fold} ---")
    
    X_tr, X_val = X_train_encoded.iloc[train_idx], X_train_encoded.iloc[valid_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]
    
    dtrain = xgb.DMatrix(X_tr, label=y_tr, enable_categorical=True)
    dvalid = xgb.DMatrix(X_val, label=y_val, enable_categorical=True)
    
    model = xgb.train(
        xgb_params,
        dtrain,
        num_boost_round=1000,
        evals=[(dtrain, 'train'), (dvalid, 'valid')],
        early_stopping_rounds=50,
        verbose_eval=200
    )
    
    dvalid_pred = xgb.DMatrix(X_val, enable_categorical=True)
    dtest_pred = xgb.DMatrix(X_test_encoded, enable_categorical=True)
    
    oof_xgb[valid_idx] = model.predict(dvalid_pred)
    test_xgb += model.predict(dtest_pred) / skf.n_splits
    
    fold_auc = roc_auc_score(y_val, oof_xgb[valid_idx])
    print(f"Fold {fold} AUC: {fold_auc:.6f}")

xgb_cv_auc = roc_auc_score(y_train, oof_xgb)
print(f"\n XGBoost CV AUC: {xgb_cv_auc:.6f}")

# ============================
# 10. 개별 모델 결과 요약
# ============================
print("\n" + "="*60)
print("INDIVIDUAL MODEL RESULTS")
print("="*60)
print(f"LightGBM CV AUC: {lgb_cv_auc:.6f}")
print(f"CatBoost CV AUC: {cat_cv_auc:.6f}")
print(f"XGBoost  CV AUC: {xgb_cv_auc:.6f}")

# ============================
# 11. Stacking (Level 2 Meta Model)
# ============================
print("\n" + "="*60)
print("STACKING ENSEMBLE")
print("="*60)

# OOF 예측값들을 Meta Feature로 사용
meta_train = np.column_stack([oof_lgb, oof_cat, oof_xgb])
meta_test = np.column_stack([test_lgb, test_cat, test_xgb])

print(f"Meta train shape: {meta_train.shape}")
print(f"Meta test shape: {meta_test.shape}")

# LogisticRegression으로 Stacking
meta_model = LogisticRegression(C=0.1, random_state=42, max_iter=1000)
meta_model.fit(meta_train, y_train)

# Meta model 예측
final_oof = meta_model.predict_proba(meta_train)[:, 1]
final_test = meta_model.predict_proba(meta_test)[:, 1]

stacking_cv_auc = roc_auc_score(y_train, final_oof)
print(f"\nStacking CV AUC: {stacking_cv_auc:.6f}")

# Meta model 가중치 확인
print(f"\nMeta Model Weights:")
print(f"  LightGBM: {meta_model.coef_[0][0]:.4f}")
print(f"  CatBoost: {meta_model.coef_[0][1]:.4f}")
print(f"  XGBoost:  {meta_model.coef_[0][2]:.4f}")

# ============================
# 12. 최종 결과 비교
# ============================
print("\n" + "="*60)
print("FINAL RESULTS SUMMARY")
print("="*60)
results_df = pd.DataFrame({
    'Model': ['LightGBM', 'CatBoost', 'XGBoost', 'Stacking'],
    'CV AUC': [lgb_cv_auc, cat_cv_auc, xgb_cv_auc, stacking_cv_auc]
}).sort_values('CV AUC', ascending=False)

print(results_df.to_string(index=False))

# ============================
# 13. Submission 생성 (Stacking 결과)
# ============================
submission['probability'] = final_test
submission.to_csv("../outputs/submission_stacking_v1(0206).csv", index=False)
print("\n Stacking Submission saved!")

# 개별 모델 submission도 저장
submission_lgb = submission.copy()
submission_lgb['probability'] = test_lgb
submission_lgb.to_csv("../outputs/submission_lgbm_v2(0206).csv", index=False)

submission_cat = submission.copy()
submission_cat['probability'] = test_cat
submission_cat.to_csv("../outputs/submission_catboost_v1(0206).csv", index=False)

submission_xgb = submission.copy()
submission_xgb['probability'] = test_xgb
submission_xgb.to_csv("../outputs/submission_xgboost_v1(0206).csv", index=False)

print("개별 모델 Submissions도 저장 완료!")

# ============================
# 14. 예측값 분포 확인
# ============================
print("\n=== 예측값 분포 (Stacking) ===")
print(f"Min:  {final_test.min():.6f}")
print(f"Max:  {final_test.max():.6f}")
print(f"Mean: {final_test.mean():.6f}")
print(f"Std:  {final_test.std():.6f}")

print(f"\n=== Test 예측 분포 ===")
print(pd.Series(final_test).describe())

Train shape: (256351, 69)
Test shape: (90067, 68)
Submission shape: (90067, 2)

전처리 후 Train shape: (256351, 86)
전처리 후 Test shape: (90067, 85)

범주형 변수 28개 인코딩 완료

MODEL 1: LightGBM

--- Fold 1 ---
Training until validation scores don't improve for 50 rounds
[200]	train's auc: 0.755447	valid's auc: 0.73754
Early stopping, best iteration is:
[152]	train's auc: 0.752014	valid's auc: 0.737696
Fold 1 AUC: 0.737696

--- Fold 2 ---
Training until validation scores don't improve for 50 rounds
[200]	train's auc: 0.754134	valid's auc: 0.741907
Early stopping, best iteration is:
[167]	train's auc: 0.751816	valid's auc: 0.742207
Fold 2 AUC: 0.742207

--- Fold 3 ---
Training until validation scores don't improve for 50 rounds
[200]	train's auc: 0.754311	valid's auc: 0.739913
Early stopping, best iteration is:
[236]	train's auc: 0.756842	valid's auc: 0.739972
Fold 3 AUC: 0.739972

--- Fold 4 ---
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[122]	trai

In [2]:
# ============================
# 11. 다양한 Ensemble 방법 비교
# ============================
from scipy.stats import rankdata

print("\n" + "="*60)
print("ENSEMBLE METHOD COMPARISON")
print("="*60)

# 1. Stacking (기존)
meta_train = np.column_stack([oof_lgb, oof_cat, oof_xgb])
meta_test = np.column_stack([test_lgb, test_cat, test_xgb])
meta_model = LogisticRegression(C=0.1, random_state=42, max_iter=1000)
meta_model.fit(meta_train, y_train)
stacking_oof = meta_model.predict_proba(meta_train)[:, 1]
stacking_test = meta_model.predict_proba(meta_test)[:, 1]
stacking_auc = roc_auc_score(y_train, stacking_oof)

# 2. Simple Averaging
simple_oof = (oof_lgb + oof_cat + oof_xgb) / 3
simple_test = (test_lgb + test_cat + test_xgb) / 3
simple_auc = roc_auc_score(y_train, simple_oof)

# 3. Weighted Averaging (CV AUC 기반)
weights = np.array([lgb_cv_auc, cat_cv_auc, xgb_cv_auc])
weights = weights / weights.sum()
weighted_oof = weights[0]*oof_lgb + weights[1]*oof_cat + weights[2]*oof_xgb
weighted_test = weights[0]*test_lgb + weights[1]*test_cat + weights[2]*test_xgb
weighted_auc = roc_auc_score(y_train, weighted_oof)

# 4. Rank Averaging
rank_oof_lgb = rankdata(oof_lgb) / len(oof_lgb)
rank_oof_cat = rankdata(oof_cat) / len(oof_cat)
rank_oof_xgb = rankdata(oof_xgb) / len(oof_xgb)
rank_oof = (rank_oof_lgb + rank_oof_cat + rank_oof_xgb) / 3

rank_test_lgb = rankdata(test_lgb) / len(test_lgb)
rank_test_cat = rankdata(test_cat) / len(test_cat)
rank_test_xgb = rankdata(test_xgb) / len(test_xgb)
rank_test = (rank_test_lgb + rank_test_cat + rank_test_xgb) / 3
rank_auc = roc_auc_score(y_train, rank_oof)

# 5. Weighted Rank Averaging (추천!)
rank_weighted_oof = weights[0]*rank_oof_lgb + weights[1]*rank_oof_cat + weights[2]*rank_oof_xgb
rank_weighted_test = weights[0]*rank_test_lgb + weights[1]*rank_test_cat + weights[2]*rank_test_xgb
rank_weighted_auc = roc_auc_score(y_train, rank_weighted_oof)

# 결과 비교
ensemble_results = pd.DataFrame({
    'Method': [
        'LightGBM', 'CatBoost', 'XGBoost',
        'Simple Avg', 'Weighted Avg', 
        'Rank Avg', 'Weighted Rank', 'Stacking'
    ],
    'CV AUC': [
        lgb_cv_auc, cat_cv_auc, xgb_cv_auc,
        simple_auc, weighted_auc, 
        rank_auc, rank_weighted_auc, stacking_auc
    ]
}).sort_values('CV AUC', ascending=False)

print(ensemble_results.to_string(index=False))

# 가중치 출력
print(f"\nWeights (based on CV AUC):")
print(f"  LightGBM: {weights[0]:.4f}")
print(f"  CatBoost: {weights[1]:.4f}")
print(f"  XGBoost:  {weights[2]:.4f}")

# 최고 성능 방법 찾기
best_method = ensemble_results.iloc[0]['Method']
print(f"\n🏆 Best Method: {best_method}")


ENSEMBLE METHOD COMPARISON
       Method   CV AUC
     Stacking 0.740306
 Weighted Avg 0.740230
   Simple Avg 0.740230
Weighted Rank 0.740228
     Rank Avg 0.740228
     CatBoost 0.739827
     LightGBM 0.739818
      XGBoost 0.738716

Weights (based on CV AUC):
  LightGBM: 0.3335
  CatBoost: 0.3335
  XGBoost:  0.3330

🏆 Best Method: Stacking
